In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import pymysql as mdb
import sqlalchemy as sa
import matplotlib.pyplot as plt 

from sklearn.cross_validation import train_test_split
from sklearn import metrics

from decimal import *

import sexmachine.detector as gender
identify_gender = gender.Detector()


In [2]:
# load local sql engine
import pickle
import sys
import nltk
import re
import psycopg2

import pyLDAvis as plda
plda.enable_notebook()
import pyLDAvis.gensim

import gensim
from sqlalchemy_utils import database_exists, create_database

%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
sns.set_context('talk')
sns.set_style('darkgrid') 
plt.rcParams['figure.figsize'] = 12, 8  # plotsize
pd.set_option('display.max_columns', 500)
local_weave_pair = sa.create_engine("postgres://%s@localhost/%s"%('rootname','weave_pair'))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sqlalchemy/dialects/__init__.py:34: SADeprecationWarning: The SQLAlchemy PostgreSQL dialect has been renamed from 'postgres' to 'postgresql'. The new URL format is postgresql[+driver]://<user>:<pass>@<host>/<dbname>
  module = __import__('sqlalchemy.dialects.%s' % (dialect, )).dialects


## Load in weave database

In [3]:
conConcierge = mdb.connect(host='alfred-prod.cbyff1yl1xje.us-west-2.rds.amazonaws.com', 
                  user='insight', 
                  password='YWRt9fxYzx1PqHy', 
                  db='alfred_prod')
curC = conConcierge.cursor()
curC.execute('SHOW TABLES')
table_names_C = curC.fetchall()

In [4]:
conWeave = mdb.connect(host='weave-prod-analytics.cbyff1yl1xje.us-west-2.rds.amazonaws.com', 
                  user='insight', 
                  password='2xMhnE458gSPxeS', 
                  db='weave_prod')
curW = conWeave.cursor()
curW.execute('SHOW TABLES')
table_names_W = curW.fetchall()

In [ ]:
print table_names_C
print table_names_W

In [5]:
def table_to_df(table,cur):
    cur.execute('SELECT * from {table}'.format(table=table))
    rows = cur.fetchall()
    df = pd.DataFrame( [[ij for ij in i] for i in rows])
    cur.execute('DESCRIBE {table}'.format(table=table))
    cols = cur.fetchall()
    dfcols = [i[0] for i in cols]
    df.columns = dfcols
    return df

In [6]:
## create a database (if it doesn't exist)
if not database_exists(local_weave_pair.url):
    create_database(local_weave_pair.url)
print(database_exists(local_weave_pair.url))

# connect:
#con = None
con = psycopg2.connect(database = 'weave_pair', user = 'jiongz')
#con = psycopg2.connect(database = dbname, user = username, password = passwd)


# query:
sql_query = """
SELECT * FROM user_info_linkedin;
"""

user_linkedin_info=pd.read_sql_query(sql_query, con)

True


### user feature selection

In [7]:
# import 6 useful tables into data frames

match_feedbacks = table_to_df('match_feedbacks',curC)
match_feedbacks = match_feedbacks.sort('user_id')

registrations = table_to_df('registrations',curC)
registrations = registrations.sort('id')

user_infos = table_to_df('user_info_stemmeds',curC)
user_infos = user_infos.sort('user_id')

#pref_question_answers = table_to_df('pref_question_answers',curC)
#pref_question_answers = pref_question_answers.sort('user_id')

educations = table_to_df('educations',curW)
educations = educations.sort('user_id')

user_tags = table_to_df('users_tags',curW)
user_tags = user_tags.sort('user_id')

### user feature table:
user_id  first_name  meeting_times  neighborhood  degree  years(after graduation)  tag_set  field  title  looking_for

In [8]:
user_id = match_feedbacks['user_id']
meeting_times = user_id.value_counts()

In [9]:
#get user feature: meeting_times
meeting_times=pd.DataFrame(np.column_stack((np.array(meeting_times.index), np.array(meeting_times))),columns=['user_id','meeting_times'])

In [10]:
# get useful dataset user_ids
user_ids = meeting_times['user_id'].values
len(user_ids)

544

trim tables according to useful user_ids

In [11]:
# update 4 user feature tables according to useful user_ids
useful_user_infos = user_infos.loc[user_infos['user_id'].isin(list(user_ids))]
useful_registrations = registrations.loc[registrations['id'].isin(list(user_ids))]
useful_educations = educations.loc[educations['user_id'].isin(list(user_ids))]
useful_user_tags = user_tags.loc[user_tags['user_id'].isin(list(user_ids))]

In [12]:
# get user feature: neigborhood
neighborhood = useful_user_infos[['user_id','hood']]

In [13]:
# get user feature: degree
degree = useful_educations.groupby(['user_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['user_id']])  ))['degree'] 
degree = pd.DataFrame(np.column_stack((np.array(degree.index), np.array(degree))),columns=['user_id','degree'])

In [14]:
# get user feature: years
years = useful_educations.groupby(['user_id'])['end_year'].max() 
years = pd.DataFrame(np.column_stack((np.array(years.index), np.array(years))),columns=['user_id','end_year'])

In [15]:
# get user feature: tags
tags = useful_user_tags.groupby(['user_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['user_id']])  ))['tag_id'] 
tags = pd.DataFrame(np.column_stack((np.array(tags.index), np.array(tags))),columns=['user_id','tags'])

In [16]:
# get user feature: title
title = useful_user_infos.groupby(['user_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['user_id']])  ))['title'] 
title = pd.DataFrame(np.column_stack((np.array(title.index), np.array(title))),columns=['user_id','title'])

In [17]:
# get user feature: looking_for
looking_for = useful_user_infos.groupby(['user_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['user_id']])  ))['looking_for'] 
looking_for = pd.DataFrame(np.column_stack((np.array(looking_for.index), np.array(looking_for))),columns=['user_id','looking_for'])

In [18]:
# get user feature: name
name = useful_registrations.groupby(['id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['id']])  ))['first_name'] 
name = pd.DataFrame(np.column_stack((np.array(name.index), np.array(name))),columns=['user_id','first_name'])

In [19]:
# get user feature: get_rating
match_feedbacks.sort('week_id')
get_ratings = match_feedbacks.groupby(['match_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['match_id']])  ))['rating'] 
get_ratings = pd.DataFrame(np.column_stack((np.array(get_ratings.index), np.array(get_ratings))),columns=['user_id','get_ratings'])
get_ratings_mean = pd.DataFrame(np.column_stack((np.array(get_ratings['user_id'].values), np.array([np.mean(a) for a in get_ratings['get_ratings'].values]))),columns=['user_id','get_ratings_mean']) 

In [20]:
# get user label: ratings
match_feedbacks.sort('week_id')
ratings = match_feedbacks.groupby(['user_id']).apply(lambda tdf: pd.Series(  dict([[vv,tdf[vv].tolist()] for vv in tdf if vv not in ['user_id']])  ))['rating'] 
ratings = pd.DataFrame(np.column_stack((np.array(ratings.index), np.array(ratings))),columns=['user_id','ratings'])
ratings_mean = pd.DataFrame(np.column_stack((np.array(ratings['user_id'].values), np.array([np.mean(a) for a in ratings['ratings'].values]))),columns=['user_id','ratings_mean']) 

In [ ]:
pd.merge(get_ratings_mean, ratings_mean, on='user_id').astype(float).plot(x='get_ratings_mean',y='ratings_mean',kind='scatter')

In [21]:
user_features_all = name.merge(meeting_times, on='user_id',how='outer').merge(neighborhood, on='user_id',how='outer').merge(degree, on='user_id',how='outer').merge(years, on='user_id',how='outer').merge(tags, on='user_id',how='outer').merge(title, on='user_id',how='outer').merge(looking_for, on='user_id',how='outer').merge(get_ratings_mean, on='user_id',how='outer').merge(ratings_mean, on='user_id',how='outer')
user_features_all.loc[user_features_all['user_id'].isin(list(user_ids))]

,user_id,first_name,meeting_times,hood,degree,end_year,tags,title,looking_for,get_ratings_mean,ratings_mean
0,2,[Andrew],1,soma,NaN,NaN,NaN,[founder coo],[],4.75,5.00
1,10,[Jeffrey],8,,"[M.B.A., BS, Doctorate of Pharmacy, High School]",2012,"[9835, 52680, 1753, 4751, 36, 29, 6087, 78993,...",[head mobil market],[],5.00,4.375
2,12,[Joe],7,,"[None, Bachelor of Science]",2010,"[0, 36, 59, 1764]",[io develop],[],4.00,4.857142857142857142857142857
3,34,[Dora],3,soma financi district union squar hay vallei,[BS],2012,"[0, 1776, 889, 1724]",[vp busi develop],[busi develop sale],4.50,3.00
4,59,[Beatriz],3,financi district,[BS],2006,"[711, 0]",[founder],[],4.50,5.00
5,90,[Jibran],3,soma financi district,"[Associate of Arts, Bachelor of Arts]",2010,"[376, 19732, 1769, 454, 0]",[founder],[investor angel smart citi media innov meta da...,3.142857142857142857142857143,4.333333333333333333333333333
6,97,[Ian],3,market st,[None],2005,"[67, 0, 1763, 1726]",[director media technolog partnership],[media startup],4.00,3.666666666666666666666666667
7,100,[Shaheen],1,financi district,"[MS, BS (Honors), SM]",2011,"[4772, 63908, 35216, 4829, 4848, 4843, 8872, 4...",[account manag],[educ startup develop sale],4.666666666666666666666666667,5.00
8,105,[Matt],2,fidi soma,[Bachelor's Degree],2007,"[86866, 0, 19732, 4879, 7136, 67344, 1763, 172...",[entrepreneur resid],[marketplac entrepreneur pm growth hacker],3.00,4.50
9,108,[Alan],1,castro soma,"[None, None]",2009,"[1138, 1111, 0, 1763, 1724, 96036, 19732]",[store manag],[cpg market brand manag recruit],NaN,5.00


In [22]:
user_features = meeting_times.merge(neighborhood, on='user_id',how='outer').merge(title, on='user_id',how='outer').merge(looking_for, on='user_id',how='outer').merge(get_ratings_mean, on='user_id',how='outer').merge(ratings_mean, on='user_id',how='outer')
user_features.loc[user_features['user_id'].isin(list(user_ids))]

,user_id,meeting_times,hood,title,looking_for,get_ratings_mean,ratings_mean
0,307,12,soma market st north beach,[ceo],[investor founder],4.333333333333333333333333333,4.25
1,2183,11,,[founder],[],4.888888888888888888888888889,4.363636363636363636363636364
2,1081,10,,[softwar engin],[],4.50,4.50
3,322,10,financi district soma union squar,[advertis account manag],[backend engin io develop softwar engin design...,3.833333333333333333333333333,3.90
4,1003,9,soma,[develop softwar engin],[startup strategi yc compani],2.60,4.333333333333333333333333333
5,1102,9,soma,[founder],[investor ui ux],3.80,4.888888888888888888888888889
6,10,8,,[head mobil market],[],5.00,4.375
7,16515,8,,[founder ceo],[angel growth],3.333333333333333333333333333,4.125
8,646,8,financi district,[advisor medic director],[medic healthcar ceo founder vc],4.25,4.25
9,17590,7,market st,[senior ux design],[design],4.00,5.00


### combine linkedin info and database info: user_linkedin_info + user_features

In [23]:
user_features_combine=user_features.merge(user_linkedin_info, on='user_id', how='outer')

### write table into sql

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
user_features.to_sql('user_features', local_weave_pair, if_exists='replace')
user_features_combine.to_sql('user_features_combine', local_weave_pair, if_exists='replace')

## build feedback event features table

In [77]:
user_ids1 = user_features['user_id'].values
user_ids2 = user_linkedin_info['user_id'].values
user_ids = set(user_ids1).intersection(set(user_ids2))
len(user_ids)

527

In [84]:
feedback_events = match_feedbacks[match_feedbacks['match_id'].isin(user_ids)]
feedback_events = feedback_events[feedback_events['user_id'].isin(user_ids)]

In [85]:
dataset_0=pd.merge(
    pd.merge(
        pd.merge(
            feedback_events[['week_id', 'rating', 'user_id', 'match_id']],
            user_features[['user_id','meeting_times']],on='user_id'),
        user_features[['user_id','meeting_times']],left_on='match_id',right_on='user_id'),
    user_features[['user_id','get_ratings_mean']],left_on='match_id',right_on='user_id')
feedback_dataset_0=dataset_0[['week_id', 'rating','user_id_x','match_id','meeting_times_x','meeting_times_y','get_ratings_mean']]    

In [86]:
feedback_dataset_0 = feedback_dataset_0.sort(['week_id'])
len(feedback_dataset_0)

963

In [91]:
user_linkedin_info = user_linkedin_info.drop(user_linkedin_info.index[510])
user_linkedin_info.head(5)

,index,user_id,name,degree,start_yr,majors
0,0,1230,Zoe Weintraub,"B.A.,",2009,"{""International Relations, Entrepreneurship, D..."
1,1,186,Sanjay Mahalingam,"ICSE,",2011,"{""Management Science and Engineering"",""Credent..."
2,2,174,Roger Brown,"Bachelor of Science (BS),",2006,"{""Product Management, Entrepreneurship & Innov..."
3,3,224,Jordan Crawford,"Bachelor's degree,",2003,"{""Political Science"",""Political Science and Go..."
4,4,1159,"Brian Samson, SPHR","Bachelor's Degree,",2002,"{""Human Resources/General Management""}"


In [92]:
# calculate match score for all meeting events, get features from linkedin info for meeting events

match_scores = []
user_degrees = []
match_degrees = []
degree_dict = {'b':1, 'm':2, 'p':3}
user_yrs = []
match_yrs = []
user_genders = []
match_genders = []
gender_dict = {'female':0, 'mostly_female':0, 'male':1, 'mostly_male':1, 'andy':2}

for index,row in feedback_dataset_0.iterrows():
    print row['user_id_x'], row['match_id']
    # calculate match score
    demands = set(user_features.loc[user_features['user_id']==row['user_id_x'],'looking_for'].values[0][0].split())
    supplies = set(user_features.loc[user_features['user_id']==row['match_id'],'title'].values[0][0].split())
    if not demands:
        match_scores.append(1)
    else:
        if demands.intersection(supplies):
            match_scores.append(2)
        else:
            match_scores.append(0)
    
    # get degree
    if user_linkedin_info.loc[user_linkedin_info['user_id']==row['user_id_x'],'degree'].values:
        user_degree = user_linkedin_info.loc[user_linkedin_info['user_id']==row['user_id_x'],'degree'].values[0][0].lower()
        if user_degree in degree_dict:
            user_degrees.append(degree_dict[user_degree])
        else:
            user_degrees.append(0)
    else:
        user_degrees.append(0)
    if user_linkedin_info.loc[user_linkedin_info['user_id']==row['match_id'],'degree'].values.any():
        match_degree = user_linkedin_info.loc[user_linkedin_info['user_id']==row['match_id'],'degree'].values[0][0].lower()
        if match_degree in degree_dict:
            match_degrees.append(degree_dict[match_degree])
        else:
            match_degrees.append(0)
    else:
        match_degrees.append(0)
        
    # get yr
    user_yrs.append(user_linkedin_info.loc[user_linkedin_info['user_id']==row['user_id_x'],'start_yr'].values[0])
    match_yrs.append(user_linkedin_info.loc[user_linkedin_info['user_id']==row['match_id'],'start_yr'].values[0])
    
    # get gender
    user_name = user_linkedin_info.loc[user_linkedin_info['user_id']==row['user_id_x'],'name'].values[0].split()[0]
    user_genders.append(gender_dict[identify_gender.get_gender(user_name)])
    match_name = user_linkedin_info.loc[user_linkedin_info['user_id']==row['match_id'],'name'].values[0].split()[0]
    match_genders.append(gender_dict[identify_gender.get_gender(match_name)])

1102 175
895 163
584 34
175 1102
434 605
939 358
163 895
34 584
451 732
129 266
990 17478
708 355
289 378
355 708
17478 990
105 444
627 1218
59 174
732 451
1218 627
627 105
100 990
1089 224
105 627
278 322
639 762
1036 360
147 422
137 776
248 895
381 267
301 705
360 1036
684 912
505 570
912 684
570 505
985 222
705 301
238 474
312 936
895 248
762 639
322 278
217 737
422 147
481 113
776 137
936 312
59 164
570 307
584 388
610 2183
1286 1817
1817 1286
10 1111
388 584
505 734
164 59
1111 10
895 762
307 570
1081 2
2543 924
1183 2183
163 358
2922 576
213 1102
653 1817
355 34
193 290
2 1081
924 2543
1102 213
584 1315
358 163
2183 1183
1111 639
1315 584
6339 5496
1193 1111
163 2
985 475
1003 6453
129 288
355 2922
531 378
1817 307
475 985
307 1817
322 90
1081 474
9470 186
5496 6339
474 1081
646 605
322 113
698 2543
1315 1286
163 475
113 322
16509 924
222 59
737 809
193 6339
399 884
990 1183
1081 378
34 266
1286 1315
290 97
297 273
605 646
2543 698
307 936
9470 689
97 290
2922 10
884 399
288 147


In [94]:
print len(match_scores)
print len(user_degrees)
print len(match_degrees)   
print len(user_yrs)
print len(match_yrs) 
print len(user_genders)
print len(match_genders)

963
963
963
963
963
963
963


In [95]:
# meeting features dataset: feedback_dataset 
feedback_dataset = feedback_dataset_0.copy()
feedback_dataset['match_scores'] = pd.Series(np.array(match_scores), index=feedback_dataset.index)
feedback_dataset['user_degrees'] = pd.Series(np.array(user_degrees), index=feedback_dataset.index)
feedback_dataset['match_degrees'] = pd.Series(np.array(match_degrees), index=feedback_dataset.index)
feedback_dataset['user_yrs'] = pd.Series(np.array(user_yrs), index=feedback_dataset.index)
feedback_dataset['match_yrs'] = pd.Series(np.array(match_yrs), index=feedback_dataset.index)
feedback_dataset['user_genders'] = pd.Series(np.array(user_genders), index=feedback_dataset.index)
feedback_dataset['match_genders'] = pd.Series(np.array(match_genders), index=feedback_dataset.index)

In [101]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
feedback_dataset.to_sql('feedback_dataset', local_weave_pair, if_exists='replace')

In [105]:
training_set = feedback_dataset[feedback_dataset['week_id']<=26]
print len(training_set)
testing_set = feedback_dataset[feedback_dataset['week_id']>26]
print len(testing_set)

698
265


In [104]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
## df is any pandas dataframe 
training_set.to_sql('training_set', local_weave_pair, if_exists='replace')
testing_set.to_sql('testing_set', local_weave_pair, if_exists='replace')

# Random Forest Model

In [ ]:
# set up  test data sets
y = dataset_0.loc[:500,'rating'].values
X = dataset_0.loc[:500,['meeting_times_x','meeting_times_y','get_ratings_mean']].values

In [ ]:
X.astype(float),y.astype(int)

In [ ]:
 # test 1 with RF

from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = RandomForestClassifier(n_estimators=100, random_state=10).fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.ensemble import RandomForestRegressor

print("accuracy:", metrics.accuracy_score(y_test.astype(int), y_pred.astype(int)))
print("precision:", metrics.precision_score(y_test.astype(int), y_pred.astype(int), average='micro'))
print("recall:", metrics.recall_score(y_test.astype(int), y_pred.astype(int), average='micro'))
print("f1 score:", metrics.f1_score(y_test.astype(int), y_pred.astype(int), average='micro'))

In [ ]:
# comparing with random
y_rand_pred = np.random.randint(1,6,len(y_test))

print("accuracy:", metrics.accuracy_score(y_test.astype(int), y_rand_pred.astype(int)))
print("precision:", metrics.precision_score(y_test.astype(int), y_rand_pred.astype(int), average='micro'))
print("recall:", metrics.recall_score(y_test.astype(int), y_rand_pred.astype(int), average='micro'))
print("f1 score:", metrics.f1_score(y_test.astype(int), y_rand_pred.astype(int), average='micro'))